In [ ]:
import pandas as pd
import re
from collections import defaultdict
import pandas.pyplot as plt
import seaborn as sns

### Import of Data

In [ ]:
df=pd.read_excel("C:/.../Enriched transcripts.xlsx", skiprows=[0])

In [ ]:
with open("C:/.../TAIR10_cds_20101214_updated") as f:
    headers=[line for line in f if ">" in line]

In [ ]:
cds=[bed(i) for i in headers]
cds=list(filter(lambda x:x!=None,cds))

cds_lims={i[3]:[i[1],i[2]]for i in cds}

In [ ]:
with open("C:/.../TAIR10_cdna_20101214_updated") as f:
    seqs=[line.strip() for line in f if ">" in line]

In [ ]:
def bed(seq):
    m=re.findall(r"([ATM]{2}\dG\d{5}\.\d+)|(chr\d)\:(\d+)-(\d+)|(REVERSE)|(FORWARD)",seq)
    data=list(map(lambda x: list(filter(lambda y: y!="", x)), m))
    if len(data)==3:
        data=list(map(lambda x: "+" if x[0]=="FORWARD" else ("-" if x[0]=="REVERSE" else x),data))
        return [f"{'C'+data[1][0][1:]}",int(data[1][1]),int(data[1][2]),data[0][0],data[2]]

In [ ]:
beds=[bed(i) for i in seqs]
beds=list(filter(lambda x:x!=None,beds))

In [ ]:
filt=list(filter(lambda x: x[3][:-2] in df[df["Log ratio mock"]>1]["geneID"].to_list(), beds))

filt_heat=list(filter(lambda x: x[3][:-2] in df[df["Log ratio heat"]>1]["geneID"].to_list(), beds))

### m6A site identification

In [ ]:
vir1_VIRc_df=pd.read_table(r"C:\...\elife-49658-fig5-data1-v1.tds")

In [ ]:
#Same code used twice// Once with filt and another time with filt_heat
#Used to find the location of the m6A sites in PB-enriched genes
ls=[]
def find_enriched(row):
    global ls
    choices=list(filter(lambda x: int(row[0])==int(x[0][-1:]) and row[5]==x[4],filt))
    if choices:
        for choice in choices:
            if choice[1]<=row[1] and row[2]<=choice[2]:
                ls+=[{"Gene":choice[3],"#chr":row[0],"start":row[1],"end":row[2],"score":row[4],"strand":row[5],"LogFC":row[6]}]
    else:
        print(":(")

vir1_VIRc_df.apply(lambda x: find_enriched(x),axis=1)

In [ ]:
enriched_sites=pd.DataFrame.from_records(ls)

enriched_sites=enriched_sites[enriched_sites["Gene"].str.contains(".1",regex=False)]

In [ ]:
heat_enriched_sites=pd.DataFrame.from_records(ls)

heat_enriched_sites=heat_enriched_sites[heat_enriched_sites["Gene"].str.contains(".1",regex=False)]

In [ ]:
#Used twice to find where the m6A sites are  
ls=[]
def find_where(row):
    global ls
    choices=list(filter(lambda x: row[0]==x[0],cds_lims.items()))
    if choices:
        for choice in choices:
            if row[5]=="+":
                if row[3]<choice[1][0]:
                    ls+=["5' UTR"]
                elif choice[1][0]<row[2]<choice[1][1] and  choice[1][0]<row[3]<choice[1][1]:
                    ls+=['CDS']
                else:
                    ls+=["3' UTR"]

            if row[5]=="-":
                if row[2]>choice[1][1]:
                    ls+=["5' UTR"]
                elif choice[1][0]<row[2]<choice[1][1] and  choice[1][0]<row[3]<choice[1][1]:
                    ls+=['CDS']
                else:
                    ls+=["3' UTR"]
    else:
        print(row[0])

In [ ]:
enriched_sites[enriched_sites["LogFC"]>1].apply(lambda x: find_where(x),axis=1)

heat_enriched_sites[heat_enriched_sites["LogFC"]>1].apply(lambda x: find_where(x),axis=1)

### Differential gene expression

In [ ]:
vir_diff=pd.read_excel(r"C:\...\elife-49658-fig6-data1-v1.xlsx")
vir_diff["gene_id"]=vir_diff["gene_id"].apply(lambda x: x+".1")

In [ ]:
not_menr=set(map(lambda x: x+".1",df[(df["Log ratio mock"]>1)]["geneID"].to_list()))-set(enriched_sites["Gene"])

not_henr=set(map(lambda x: x+".1",df[(df["Log ratio heat"]>1)]["geneID"].to_list()))-set(heat_enriched_sites["Gene"])

In [ ]:
m6a_yes=vir_diff[vir_diff["gene_id"].isin(enriched_sites["Gene"].to_list())]["logFC"].to_list()

m6a_no=vir_diff[vir_diff["gene_id"].isin(not_menr)]["logFC"].to_list()



m6a_yes_h=vir_diff[vir_diff["gene_id"].isin(heat_enriched_sites["Gene"].to_list())&(vir_diff["FDR"]<0.05)]["logFC"].to_list()

m6a_no_h=vir_diff[vir_diff["gene_id"].isin(not_henr)&(vir_diff["FDR"]<0.05)]["logFC"].to_list()

In [ ]:
fig, ax=plt.subplots()
sns.kdeplot(m6a_yes_h,fill=True)
sns.kdeplot(m6a_no_h,fill=True)

ax.legend(labels=["Genes with error sites","Genes with no error sites"])
ax.set_xlabel("vir1/VIRc LogFC")
ax.set_title("Heat PB-in")

#ax.set_xlim(-2.5, 2.5)
#plt.savefig("name.tiff", dpi=600)